In [1]:
import numpy as np
import pandas as pd 
def precisionMatrix(proba, y):
    def _precisionClassify(df,proba, wins, c =0 ):
        for indx, v in enumerate(proba):
            row = 0
            col = 0
            if wins[indx] == c:
                col = 0
            else:
                col =1
            if v <0.2:
                row =6 
            elif v < 0.3 and  v >=0.2:
                row =5 
            elif v < 0.4 and v >= 0.3:
                row = 4 
            elif v < 0.5 and v >= 0.4:
                row = 3 
            elif v < 0.6 and v >= 0.5:
                row = 2 
            elif v < 0.8 and v >= 0.6:
                row = 1
            df.iloc[row,col] = df.iloc[row,col]+1 
        df[df.columns[2]] = df[df.columns[0]] /(df[df.columns[1]] + df[df.columns[0]])
    return df
    rowHeader = ['>80','60-80','50-60','40-50','30-40','20-30','<20']
    df = pd.DataFrame(np.zeros(shape=(7,3)),index=rowHeader, columns=['h_Correct', 'h_Wrong','h_Precent'])
    hproba = proba[:,0]
    wins = np.argmax(y,axis=1)
    df = _precisionClassify(df,hproba,wins)
    temp = pd.DataFrame(np.zeros(shape=(7,3)),index=rowHeader, columns=['d_Correct', 'd_Wrong','d_Precent'])
    dproba = proba[:,1]
    df = df.join(_precisionClassify(temp,dproba,wins,c=1))
    temp = pd.DataFrame(np.zeros(shape=(7,3)),index=rowHeader, columns=['a_Correct', 'a_Wrong','a_Precent'])
    aproba = proba[:,2]
    df = df.join(_precisionClassify(temp,aproba,wins,c=2))
    
    bound = pd.DataFrame(np.array([[0.8,1.0],[0.6,0.8],[0.5,0.6],[0.4,0.5],[0.3,0.4],[0.2,0.3],[0,0.2]] )
                                ,index=rowHeader, columns=['[lower', 'upper)'])
            
    return bound.join(df)
       

In [2]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
class FootballDataHelper:
    def __init__ (self, recentNum=5):
        self.win_mapping = {'H':0, 'D':1,'A':2}
        self.recentNum = recentNum
        self.df = None
        self.teamsData={}
        self.session = 0
        #self.hiddensCount = 2
       
        
    def readFootBallData(self,filename): 
        df = pd.read_csv(filename)
        #df = df.drop(df.columns[range(23,df.shape[1])], axis=1)
        #df = df.drop("Div",axis=1)
        df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)
        df['session'] = pd.Series(np.ones(shape=(df.shape[0],))*self.session, index=df.index)
        self.session = self.session +1
        #df= df.drop('Referee', 1)
        #self.team = df['HomeTeam'].drop_duplicates()
        if self.df is None:
            self.df = df
        else:
            self.df = pd.concat([self.df,df])
            
        self.df = self.df.reset_index(drop=True)
        teams = self.df['HomeTeam'].drop_duplicates()
        teamMap = {}
        for index , v in enumerate(teams):
            teamMap[v] = index
        self.teamsMap = teamMap
        referees = self.df['Referee'].drop_duplicates()
        refereesMap = {}
        for index , v in enumerate(referees):
            refereesMap[v] = index+1

        self.refereesMap = refereesMap
 
    def getTeam(self,dataFrame, teamName):       
        return dataFrame[(dataFrame["HomeTeam"] == teamName) | (dataFrame["AwayTeam"] == teamName)]
 
        
    def previousRecords(self,team, date , recentNum):
        prev = team[( team["Date"] < date)]
        
        if prev.shape[0] < recentNum :
            #print("less than min Num")
            return None
        else:
            return prev.iloc[-recentNum:]
    def readPredict (self, filename):
        df = pd.read_csv(filename)
        df['Date'] = pd.to_datetime(df['Date'],dayfirst=True)

        return df
    
    
  
    def inverseTeamMapping (self, col):
        inverseMap ={}
        for name in self.teamsMap.keys():        
            inverseMap[self.teamsMap[name]] = name
        res =[]
        for idex, v in enumerate(col):
            res.append(inverseMap[v])
        return res


    def readTeamMatch(self, teamName):
        df = pd.read_csv('teams/'+teamName+'.csv')
        df['1'] = pd.to_datetime(df['1'],yearfirst=True)
        #df['1']= (pd.to_numeric(df['1'])/1e9/24/60/60)
        self.teamsData[teamName]=df.sort(['1'],ascending=[False])
        self.teamsById[self.teamsMap[teamName]]=self.teamsData[teamName]
    
    def commonMapping(self, X):
        X['HomeTeam'] = X['HomeTeam'].map(self.teamsMap)
        X['AwayTeam'] = X['AwayTeam'].map(self.teamsMap)
        X['Referee']=X['Referee'].map(self.refereesMap).fillna(0)
        X['HTR'] = X['HTR'].map(self.win_mapping)
        X['FTR'] = X['FTR'].map(self.win_mapping)
        return X
    def initData(self, X, target,encode):
        X  = X.sort_values(by="Date")
        isInput = False
        if target is None:
            target =X      
        else:
            if self.ohe is None:
                raise Exception("Not yet get train data")
            isInput = True
            if encode == True:
                target = self.commonMapping(target)
        y=None
        if encode == True:    
            X =self.commonMapping(X)
            y = []
            for v in target['FTR']:
                y.append(range(3)==v)
        else:
            y = target['FTR'].values
        target_date = (pd.to_numeric(target['Date'])/1e9/24/60/60).values
        return isInput, X,y, target, target_date
   
    def aggregate(self,recents,nonExpand,isInput,encode):
        res =None
        if encode == True:
            if isInput==False:
                self.ohe = OneHotEncoder(categorical_features='all')
                self.ohe.fit(recents)
            res = self.ohe.transform(recents).toarray()
        else:
            res = np.array(recents)
        self.dateColumn = res.shape[1]
        res = np.hstack([res,nonExpand])
        return res
  
    def getH7(self,removeInsufficient=False, target=None,encode = True):
        #Simple recent win,draw, lose 
        df = self.df
        if removeInsufficient == True:
            df= df[df['Sufficient'] == 1]
        
        
        isInput, X, y,target, target_date = self.initData(df,target,encode)
        resy=[]
        resx=[]
        print("start format")
        recents = X[['HomeTeam','AwayTeam','Referee']].values
        haccp = X['HAccP'].values.reshape(X.shape[0],1)
        aaccp = X['AAccP'].values.reshape(X.shape[0],1)
        homeRecent = np.hstack([X[['HWin','HDraw','HLose']].values,
                                (X['HScore'].values - X['HConcede'].values).reshape(X.shape[0],1)])
        awayRecent = np.hstack([X[['AWin','ADraw','ALose']].values,
                                (X['AScore'].values - X['AConcede'].values).reshape(X.shape[0],1)])
        homeMoral = X['HMoral'].values.reshape(X.shape[0],1)
        awayMoral = X['AMoral'].values.reshape(X.shape[0],1)
        target_date = target_date.reshape(X.shape[0],1)
        nonExpand =np.hstack([target_date,X[['HRestDay','ARestDay','HAccS','AAccS','HAccST','AAccST']].values,haccp-aaccp,(haccp+1)/(aaccp+1),
                                homeRecent,awayRecent, homeMoral - awayMoral + haccp - aaccp])
        res = self.aggregate(recents,nonExpand,isInput,encode)
        print("finish")
        sys.stdout.flush()
        return res, np.array(y)
    def _getRank(self,x, X,teamName,recentNum):
        team = self.getTeam(X,teamName)
        prev = team[team['Date'] < x['Date']].values      
        for i in range(recentNum):
            pass
    def initRanking(self, n = 20):
        defaultPt = 1
        df = self.df.sort(columns=["Date"],ascending=[False])
        df["HPoints"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["APoints"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HAccP"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AAccP"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        hpoints= df["HPoints"].values
        apoints=df["APoints"].values
        ftr = df["FTR"].values
        for i in range(df.shape[0]):
            sys.stdout.write("\r progress {}".format(i))
            sys.stdout.flush()
            if ftr[i] == 'H':
                hpoints[i] = 3
                apoints[i] = 0
            elif ftr[i] == 'D':
                hpoints[i] = 1
                apoints[i] = 1
            else :
                hpoints[i] = 0
                apoints[i] = 3
        df["HPoints"]=hpoints
        df["APoints"]=apoints
        for teamName in self.teamsMap.keys():
            print(teamName)
            team  = df[(df['HomeTeam']==teamName) | (df['AwayTeam'] == teamName)] 
            hometeam = team['HomeTeam'].values
            hpoints = team['HPoints'].values
            apoints = team['APoints'].values
            psum = 0
            haccp = team['HAccP'].values
            aaccp = team['AAccP'].values
        
            for  i in range(0,n):
                if i < hpoints.shape[0]:
                    psum = psum + (hpoints[i] if hometeam[i] == teamName else apoints[i] ) 
                else:
                    psum = psum + defaultPt        
                    
        
            for j in range(team.shape[0]):

                if j+n < hpoints.shape[0]:                     
                    psum = psum + (hpoints[j+n] if hometeam[j+n]==teamName else apoints[j+n])
                else:
                    psum = psum + defaultPt 
                
                psum = psum - (hpoints[j] if hometeam[j]==teamName else apoints[j])
                    
                if hometeam[j] == teamName:
                    haccp[j]=psum
                else:
                    aaccp[j]=psum
            team['HAccp']=haccp
            team['AAccP']=aaccp
            #print(team[['HomeTeam','AwayTeam','HAccP','AAccP']])
            df.update(team)
            
            #print(df[['HomeTeam','AwayTeam','HAccP','AAccP']])
        self.df =df
        return df
    def initRecentData(self, n =5):
        df = self.df.sort(columns=["Date"],ascending=[False])
        df["HWin"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AWin"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HDraw"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["ADraw"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HLose"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["ALose"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HScore"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AScore"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HConcede"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AConcede"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HMoral"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AMoral"] = pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HRestDay"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        df["ARestDay"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HAccS"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AAccS"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        df["HAccST"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        df["AAccST"]= pd.Series(np.zeros(shape=(df.shape[0],)))
        
        df["Sufficient"] = pd.Series(np.ones(shape=(df.shape[0],)))
        
        hscore = df['FTHG'].values
        ascore = df['FTAG'].values
        hconcede = df ['FTAG'].values
        aconcede = df['FTHG'].values
        hwin = df['HWin'].values
        awin = df['AWin'].values
        hlose = df['HLose'].values
        alose = df['ALose'].values
        hdraw = df['HDraw'].values
        adraw = df['ADraw'].values
        hmoral = df['HMoral'].values
        amoral = df['AMoral'].values
        
        rankRatio = (df['HAccP'].values+1) / (df['AAccP'].values +1)
        
        
        ftr = df["FTR"].values
        for i in range(df.shape[0]):
            sys.stdout.write("\r progress {}".format(i))
            sys.stdout.flush()
            if ftr[i] == 'H':
                hwin[i] = 1
                hmoral[i] = 3 * 1/rankRatio[i]
                alose[i]= 1
                amoral[i] = -3 * 1/rankRatio[i]
            elif ftr[i] == 'D':
                hdraw[i] = 1
                hmoral[i] = 1 * 1/rankRatio[i]
                adraw[i] = 1
                amoral[i] = 1 * rankRatio[i]
            else :
                hlose[i] = 1
                hmoral[i] = -3*rankRatio[i]
                awin [i] = 1
                amoral[i] = 3*rankRatio[i]
        
        
        df["HWin"]=hwin
        df["AWin"]=awin
        df["HDraw"]=hdraw
        df["ADraw"]=adraw
        df["HLose"]=hlose
        df["ALose"]=alose
        df["HScore"]=hscore
        df["AScore"]=ascore
        df["HConcede"]=hconcede
        df["AConcede"]=aconcede
        df["HMoral"] = hmoral
        df["AMoral"] = amoral
        
        
        
        for teamName in self.teamsMap.keys():
            print(teamName)
            team  = df[(df['HomeTeam']==teamName) | (df['AwayTeam'] == teamName)] 
            hometeam = team['HomeTeam'].values
            hwin = team["HWin"].values
            awin = team["AWin"].values
            hlose= team["HLose"].values
            alose = team["ALose"].values
            hdraw = team["HDraw"].values
            adraw = team["ADraw"].values
            hscore = team["HScore"].values
            ascore = team["AScore"].values
            hconcede = team["HConcede"].values
            aconcede = team["AConcede"].values
            hmoral = team["HMoral"].values
            amoral = team["AMoral"].values
            hrestday = team["HRestDay"].values
            arestday = team["ARestDay"].values
            
            hs= team["HS"].values
            as_ = team["AS"].values
            hst = team["HST"].values
            ast = team["AST"].values
            haccs = team["HAccS"].values
            aaccs = team["AAccS"].values
            haccst =team["HAccST"].values
            aaccst = team["AAccST"].values
            
            matchDate =team['Date'].values
            sufficient = team['Sufficient'].values
            teamMatchesDate = self.teamsData[teamName].sort('1',ascending=False)['1'].values
            
            restday = 0
            winsum =0 
            losesum=0
            drawsum=0
            scoresum =0
            concedesum=0
            accs = 0
            accst = 0
            moralsum = 0
            #print(team[['HomeTeam','AwayTeam','HWin']])
            for  i in range(0,n):
                if i < team.shape[0]:
                    scoresum = scoresum + (hscore[i] if hometeam[i] == teamName else ascore[i])
                    winsum = winsum + (hwin[i] if hometeam[i] == teamName else awin[i])
                    losesum= losesum + (hlose[i] if hometeam[i] == teamName else alose[i])
                    drawsum= drawsum + (hdraw[i] if hometeam[i] == teamName else adraw[i])
                    concedesum = concedesum+ (hconcede[i] if hometeam[i] == teamName else aconcede[i])
                    moralsum= moralsum+ (hmoral[i] if hometeam[i] == teamName else amoral[i])
                    accs = accs + (hs[i] if hometeam[i] == teamName else as_[i])
                    accst = accst + (hst[i] if hometeam[i] == teamName else ast[i])
                else:
                    # + 0
                    pass
            dateIndx = 0
            for j in range(team.shape[0]):
                while True:
                    if dateIndx >= teamMatchesDate.shape[0]:
                        sufficient[j] = False
                        break
                    if teamMatchesDate[dateIndx] < matchDate[j] :
                        restday = (matchDate[j] - teamMatchesDate[dateIndx])/np.timedelta64(1,'D')
                        break
                    else:
                        dateIndx = dateIndx + 1
                
                if j+n < team.shape[0]:                     
                    scoresum = scoresum + (hscore[j+n] if hometeam[j+n] == teamName else ascore[j+n])
                    winsum = winsum + (hwin[j+n] if hometeam[j+n] == teamName else awin[j+n])
                    losesum= losesum + (hlose[j+n] if hometeam[j+n] == teamName else alose[j+n])
                    drawsum= drawsum + (hdraw[j+n] if hometeam[j+n] == teamName else adraw[j+n])
                    concedesum = concedesum+ (hconcede[j+n] if hometeam[j+n] == teamName else aconcede[j+n])
                    moralsum= moralsum+ (hmoral[j+n] if hometeam[j+n] == teamName else amoral[j+n])
                    accs = accs + (hs[j+n] if hometeam[j+n] == teamName else as_[j+n])
                    accst = accst + (hst[j+n] if hometeam[j+n] == teamName else ast[j+n])
                else:
                    sufficient[j] = False
                    
                
                scoresum = scoresum - (hscore[j] if hometeam[j] == teamName else ascore[j])
                winsum = winsum - (hwin[j] if hometeam[j] == teamName else awin[j])
                losesum= losesum - (hlose[j] if hometeam[j] == teamName else alose[j])
                drawsum= drawsum - (hdraw[j] if hometeam[j] == teamName else adraw[j])
                concedesum = concedesum - (hconcede[j] if hometeam[j] == teamName else aconcede[j])
                moralsum= moralsum - (hmoral[j] if hometeam[j] == teamName else amoral[j])
                accs = accs - (hs[j] if hometeam[j] == teamName else as_[j])
                accst = accst - (hst[j] if hometeam[j] == teamName else ast[j])
                    
                if hometeam[j] == teamName:
                    hscore[j] = scoresum
                    hwin[j] = winsum
                    hlose[j] = losesum
                    hdraw[j] = drawsum
                    hconcede[j] = concedesum
                    hmoral[j] = moralsum
                    hrestday[j] = restday
                    haccs[j]=accs
                    haccst[j]=accst
                else:
                    ascore[j] = scoresum
                    awin[j] = winsum
                    alose[j] = losesum
                    adraw[j] = drawsum
                    aconcede[j] = concedesum
                    amoral[j] = moralsum
                    arestday[j] = restday
                    aaccs[j] = accs
                    aaccst[j] = accst
            team["HWin"]=hwin
            team["AWin"]=awin
            team["HDraw"]=hdraw
            team["ADraw"]=adraw
            team["HLose"]=hlose
            team["ALose"]=alose
            team["HScore"]=hscore
            team["AScore"]=ascore
            team["HConcede"]=hconcede
            team["AConcede"]=aconcede
            team["HMoral"] = hmoral
            team["AMoral"] = amoral
            team['Sufficient'] = sufficient
            team['HAccS'] = haccs
            team['AAccS'] = aaccs
            team['HAccST'] = haccst
            team['AAccST'] = aaccst
        
            #print(team[['HomeTeam','AwayTeam','HWin']])
            df.update(team)
            #print(df[['HomeTeam','AwayTeam','HAccP','AAccP']])
        self.df =df
        return df
    def initTeamData(self):
        self.teamsData={}
        self.teamsById={}
        for name in self.teamsMap.keys():
            self.readTeamMatch(name)
            

/home/y/scikit-learn/sklearn/cross_validation.py:43: DeprecationWarning: This module has been deprecated in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
def createModel(hidSize, inputDim):
    model = Sequential()
    model.add(Dense(hidSize[0], input_dim=inputDim, init='glorot_normal'))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))

    model.add(Dense(hidSize[1], init='glorot_normal'))
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.5))

    model.add(Dense(3, init='glorot_normal'))
    model.add(Activation('softmax'))
    sgd = SGD(lr=1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer='adadelta')
    return model
earlyCallback = EarlyStopping(patience=20,verbose=1)